### Notes
Data wrangling for getting titles from crossref.org metadata.

In [8]:
import json
import pandas as pd
import numpy as np
import os

# Get directory information for file processing
cwd = os.getcwd()
dir_path = cwd + "/data/"
files = os.listdir(dir_path)



In [9]:
# Read random file and peek at data
df = pd.read_json(dir_path + '86.json')
df

,items
0,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."
1,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."
2,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."
3,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."
4,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."
...,...
2995,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."
2996,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."
2997,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."
2998,"{'indexed': {'date-parts': [[2020, 3, 25]], 'd..."


In [10]:
# Look at random row to see names of columns
row = df.loc[2222, 'items']

for key in row.keys():
    print(key)

indexed
reference-count
publisher
issue
content-domain
short-container-title
published-print
DOI
type
created
page
source
is-referenced-by-count
title
prefix
volume
member
container-title
original-title
link
deposited
score
subtitle
short-title
issued
references-count
URL
relation
ISSN
issn-type


In [11]:
# Look at title of above random row
row['title'][0]

'THE OPERATIVE TREATMENT OF TYPHOID PERFORATION.'

In [13]:
# Define acceptable characters in title
import re
pattern = r"[^A-Z0-9?().:;' ]"

# Initialize variables
total = 0
titles = ""
lens = []

# Batch process titles into single string
for file in files:
    if file.endswith('.json'):
        df = pd.read_json(dir_path + file)
    for i in range(len(df)):
        row = df.loc[i, 'items']
        title = row['title']
        title_upper = title[0].upper()
        if re.search(pattern, title_upper) is None and title_upper != "BOOKS RECEIVED" and "TEXTBOOK" not in title_upper and "CORRECTION" not in title_upper:
            titles = titles + title_upper + "+"
            total += 1
            lens.append(len(title_upper))


In [14]:
print(f"Total number of titles: {total}")
print(f"Max title length: {max(lens)}")
print(f"Min title length: {min(lens)}")
print(f"Average title length: {int(sum(lens)/len(lens))}")


Total number of titles: 8683
Max title length: 203
Min title length: 5
Average title length: 48


In [15]:
# Save titles
with open('titles.txt', "w") as file:
    file.write(titles)